In [1]:
import os
from pathlib import Path
import operator

import networkx as nx

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day21.txt'

# Part One

Build a simple graph, then the reverse topological sort well guarantee he have the integers when equations need them.

In [2]:
DG = nx.DiGraph()

G = {}

with open(FOLDER / in_file) as f:
    for line in f:
        key, job = line.strip().split(':')
        
        if job.strip().isdecimal():
            G[key] = int(job)
            DG.add_node(key, val=int(job))
        else:
            a, op, b = job.split()
            DG.add_node(key, op=op)
            DG.add_edge(key, a)
            DG.add_edge(key, b)            

            G[key] = (a, b, op)
            
ops = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': operator.truediv,
} 


for n in reversed(list(nx.topological_sort(DG))):
    if not isinstance(G[n], int):
        a, b, op = G[n]
        G[n] = ops[op](G[a],G[b]) 
        
int(G['root'])

10037517593724

# Part 2

So, using Sympy here feels a little dirty...but if just seemed better than making an abstract syntax tree (or finding some non-obvious simpler solution tonight)

In [5]:
from sympy import symbols, solve, Integer, Eq

DG = nx.DiGraph()

with open(FOLDER / in_file) as f:
    for line in f:
        key, job = line.strip().split(':')

        if job.strip().isdecimal():
            DG.add_node(key, val=int(job))
        else:
            a, op, b = job.split()
            DG.add_node(key, op=op, r=a, l=b)
            DG.add_edge(key, a)
            DG.add_edge(key, b)            

            
S = {}

for n in reversed(list(nx.topological_sort(DG))):
    node = DG.nodes[n]
    if 'val' in node:
        S[n] = symbols('humn') if n == 'humn' else Integer(node['val'])
    else:
        a, b, op =  node['r'], node['l'], node['op']
    
        S.setdefault(a,symbols(a))
        S.setdefault(b,symbols(b))
            
        f = Eq if n == 'root' else ops[op]
        S[n] = f(S[a], S[b])     
        

solve(S['root'], S['humn'])


[3272260914328]